In [2]:
# ======================================================================
# ANALISIS ULANG KETERSEDIAAN AIR (WATER YIELD) 2020 (METODE BARU)
# ======================================================================

# --- Langkah 1 & 2: Setup, AOI, dan Presipitasi ---
import ee
import geemap
ee.Authenticate()
ee.Initialize()
Map = geemap.Map()

path_aset_lc_2020 = "users/ivan_taslim/LC_Paguyaman_2020_DW_Generated"
lc_2020 = ee.Image(path_aset_lc_2020)
aoi = lc_2020.geometry()
Map.addLayer(aoi, {'color': 'blue'}, 'Batas DAS Paguyaman (AOI)')
Map.centerObject(aoi, 11)

print("Info: Menggunakan data Presipitasi tahun 2020 untuk analisis 2020.")
chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD')
precipitation_2020 = chirps.filter(ee.Filter.date('2020-01-01', '2020-12-31')).sum().clip(aoi)

# --- Langkah 3: Menghitung Evapotranspirasi 2020 dari ERA5-Land ---
print("Info: Menggunakan data ET dari ERA5-Land untuk tahun 2020.")
era5_et = ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY_AGGR').filter(ee.Filter.date('2020-01-01', '2020-12-31'))

# Pilih band evapotranspirasi total dan jumlahkan untuk setahun
# Nilai asli dalam meter, kita konversi ke milimeter
total_et_band = era5_et.select('total_evaporation_sum')
et_annual_sum_2020 = total_et_band.sum().multiply(1000).clip(aoi)


# --- Langkah 4: Menghitung Water Yield 2020 ---
print("Menghitung Water Yield dan Volume Total...")
water_yield_2020 = precipitation_2020.subtract(et_annual_sum_2020).rename('water_yield')
aoi_area_m2 = aoi.area().getInfo()
mean_yield_mm = water_yield_2020.reduceRegion(
    reducer=ee.Reducer.mean(), geometry=aoi, scale=30, maxPixels=1e12
).get('water_yield').getInfo()
mean_yield_m = mean_yield_mm / 1000
total_water_yield_volume_2020 = aoi_area_m2 * mean_yield_m

# Menampilkan hasil
print("======================================================================")
print("HASIL ANALISIS ULANG KETERSEDIAAN AIR (WATER YIELD) TAHUN 2020")
print("----------------------------------------------------------------------")
print(f"Rata-rata Water Yield: {mean_yield_mm:.2f} mm/tahun")
print(f"Total Volume Air Tersedia: {total_water_yield_volume_2020:,.2f} meter kubik/tahun")
print("======================================================================")

Map

Info: Menggunakan data Presipitasi tahun 2020 untuk analisis 2020.
Info: Menggunakan data ET dari ERA5-Land untuk tahun 2020.
Menghitung Water Yield dan Volume Total...
HASIL ANALISIS ULANG KETERSEDIAAN AIR (WATER YIELD) TAHUN 2020
----------------------------------------------------------------------
Rata-rata Water Yield: 3743.97 mm/tahun
Total Volume Air Tersedia: 14,160,597,285.70 meter kubik/tahun


Map(center=[0.7139019338201237, 122.37991662440572], controls=(WidgetControl(options=['position', 'transparent…

In [3]:
# ======================================================================
# EKSPOR PETA WATER YIELD 2020
# ======================================================================

# Ekspor ke GEE Assets
task_asset = ee.batch.Export.image.toAsset(
  image=water_yield_2020.toFloat(), # Konversi ke float untuk ekspor
  description='WaterYield_Paguyaman_2020Fixed_to_Asset',
  assetId='users/ivan_taslim/WaterYield_Paguyaman_2020Fixed', # Path aset tujuan Anda
  region=aoi,
  scale=30,
  maxPixels=1e12
)
task_asset.start()
print("Proses ekspor ke GEE ASSETS telah dimulai.")

# Ekspor ke Google Drive
task_drive = ee.batch.Export.image.toDrive(
  image=water_yield_2020.toFloat(),
  description='WaterYield_Paguyaman_2020Fixed_to_Drive',
  folder='Hasil_Analisis_DAS_Paguyaman',
  fileNamePrefix='WaterYield_Paguyaman_2020Fixed',
  region=aoi,
  scale=30,
  maxPixels=1e12
)
task_drive.start()
print("Proses ekspor ke GOOGLE DRIVE telah dimulai.")
print("\nPERIKSA TAB 'TASKS' DI GEE CODE EDITOR UNTUK MENJALANKAN KEDUA PROSES EKSPOR.")

Proses ekspor ke GEE ASSETS telah dimulai.
Proses ekspor ke GOOGLE DRIVE telah dimulai.

PERIKSA TAB 'TASKS' DI GEE CODE EDITOR UNTUK MENJALANKAN KEDUA PROSES EKSPOR.
